## Convert from XLSX -> Pandas dataframe

In [109]:
from openpyxl import load_workbook
from pandas import DataFrame
import json
import subprocess

import numpy as np
from openpyxl import load_workbook
from pandas import DataFrame

# MISSING_VALUE = "NA"
data = load_workbook('laws-influences.xlsx').active.values
header = next(data)
df = DataFrame(data, columns=header)
df['Origin ID'] = df['Origin ID'].astype("Int64")
# df.fillna(value=MISSING_VALUE, inplace=True)
# df = df.applymap(lambda v: v.strip() if type(v) is str else v)
df.set_index('Law ID', inplace=True, verify_integrity=True)

# Check for dangling nodes (no child & no parent)
roots = set(df[df['Origin ID'].isnull()].index)
leaf_nodes = set(df.index) - set(df['Origin ID'])
dangling = roots & leaf_nodes
if len(dangling) > 0:
    print(f"[WARNING] Found dangling nodes: {dangling}")

[WARNING] Found dangling nodes: {39, 40, 41, 43, 20}


In [103]:
set(df.index)

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69}

In [101]:
# How many laws repeated another one word for word?
# TODO Origin ID is missing entries
df[
    ~df['Law of Origin'].isnull() 
].groupby('Connection with Previous Law').aggregate(np.size)['Origin ID']

Connection with Previous Law
Explicit Duplication     9
Identical                6
Similar Content         33
Name: Origin ID, dtype: Int64

In [ ]:
# How many laws repeated another one word for word?
df[
    ~df['Origin ID'].
].groupby('Connection with Previous Law').aggregate(np.size)['Origin ID']

In [92]:
# How many connections crossed the Indian Ocean/Atlantic boundary?
df[~df['Origin ID'].isnull()].join(
    df, on='Origin ID', rsuffix="_origin"
).groupby(['Location_origin', 'Location']).aggregate(np.size)['Name of the Law']

Location_origin                Location                     
Caribbean                      Guyana                           2
                               India                            3
                               Isle Bourbon & Isle of France    5
                               Louisiana                        1
                               Madagascar                       2
                               New France                       2
                               Senegal                          2
                               Senegal                          1
India                          Isle Bourbon & Isle of France    1
                               Louisiana                        1
Isle Bourbon & Isle of France  Caribbean                        1
                               India                            1
                               Louisiana                        1
Louisiana                      Caribbean                        3
               

In [95]:
total_crossing = 3+5+2+1+1+1+2+1+1+1+1+1
print(f"{total_crossing} influences are crossing the IO/Atlantic boundary")

20 influences are crossing the IO/Atlantic boundary
